In [12]:
# Instalación de librerías necesarias
!pip install -U transformers
!pip install torch

In [30]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pandas as pd
import os

# 📂 Carga de historias clínicas desde Google Drive

Se monta Google Drive en el entorno de ejecución para acceder a los archivos de texto que contienen historias clínicas.

Cada archivo representa una historia clínica individual y se procesa línea por línea. Solo se almacenan las líneas no vacías, que corresponden a oraciones clínicas relevantes.

Las historias se almacenan en un diccionario llamado `historias`, donde:
- La **clave (`patient_id`)** es un número identificador único por paciente (correspondiente al archivo).
- El **valor** es una lista con las frases extraídas de ese archivo.

Esto permite estructurar y mantener el seguimiento de la procedencia de cada oración durante el análisis posterior con los modelos de NER y detección de negación/incertidumbre.


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import os
carpeta_historias = "/content/drive/MyDrive/Analitica en Salud/Negación"

In [33]:
archivos = os.listdir(carpeta_historias)
print(len(archivos))

106


In [34]:
historias = {}
for idx, archivo in enumerate(archivos, start=1):
    with open(os.path.join(carpeta_historias, archivo), "r") as f:
        lineas = [l.strip() for l in f if l.strip()]
        historias[idx] = lineas

In [35]:
len(historias)

106

In [23]:
from huggingface_hub import login
login()

# 🤖 Carga del modelo NER de cáncer de mama

Se carga el modelo preentrenado `anvorja/breast-cancer-biomedical-ner-sp-1` desde Hugging Face, diseñado para reconocer entidades biomédicas en textos relacionados con cáncer de mama. Se inicializa un pipeline de reconocimiento de entidades nombradas (NER), utilizando una estrategia de agregación `"simple"` para combinar tokens subpalabra en entidades completas.

In [36]:
# Modelo NER de cáncer de mama
ner_model_id = "anvorja/breast-cancer-biomedical-ner-sp-1"

ner_pipe = pipeline(
    "ner",
    model=AutoModelForTokenClassification.from_pretrained(ner_model_id),
    tokenizer=AutoTokenizer.from_pretrained(ner_model_id),
    aggregation_strategy="simple"
)


config.json:   0%|          | 0.00/3.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


# 🧠 Extracción de entidades clínicas con modelo NER

Se utiliza el modelo `anvorja/breast-cancer-biomedical-ner-sp-1` para identificar entidades clínicas relevantes dentro de cada oración de las historias clínicas.

Por cada oración (línea de texto) se realiza lo siguiente:

- Se aplica el pipeline NER (`ner_pipe`) para detectar entidades como biomarcadores, diagnósticos, tratamientos, etc.
- Cada entidad reconocida se registra en una estructura de tipo diccionario, guardando:
  - `patient_id`: ID del paciente correspondiente (basado en el archivo origen).
  - `sentence`: La oración en la que se detectó la entidad.
  - `NER`: El texto de la entidad reconocida.
  - `NER_label`: El tipo o categoría de la entidad según el modelo (por ejemplo: `BIOMARKER`, `TREATMENT`, etc.).

El resultado se acumula en la lista `filas_ner`, que posteriormente servirá como base para integrar el análisis de negación e incertidumbre.


In [42]:
# 2. Detectar entidades y registrar patient_id, sentence, NER + etiqueta
filas_ner = []

for patient_id, oraciones in historias.items():
    for sentence in oraciones:
        entidades = ner_pipe(sentence)
        for ent in entidades:
            filas_ner.append({
                "patient_id": patient_id,
                "sentence":   sentence,
                "NER":        ent["word"],                 # texto de la entidad
                "NER_label":  ent.get("entity_group")      # tipo de entidad
                                or ent.get("entity")
            })

In [105]:
for fila in filas_ner:
    if fila['patient_id'] == 9:
        print(fila)

{'patient_id': 9, 'sentence': 'Paciente de 37 años a la que se le realizo en junio de 2011 una ecografía mamaria por notarse una tumoración en mama derecha objetivándose una lesión de 16 x 7 mm.', 'NER': '37 años', 'NER_label': 'AGE'}
{'patient_id': 9, 'sentence': 'Paciente de 37 años a la que se le realizo en junio de 2011 una ecografía mamaria por notarse una tumoración en mama derecha objetivándose una lesión de 16 x 7 mm.', 'NER': 'junio de 2011', 'NER_label': 'DATE'}
{'patient_id': 9, 'sentence': 'Paciente de 37 años a la que se le realizo en junio de 2011 una ecografía mamaria por notarse una tumoración en mama derecha objetivándose una lesión de 16 x 7 mm.', 'NER': 'tumoración en mama derecha', 'NER_label': 'CANCER_CONCEPT'}
{'patient_id': 9, 'sentence': 'En agosto de 2011 se realiza BAG: incluye un lobulillo con fibrosis, sin otras lesiones significativas.', 'NER': 'agosto de 2011', 'NER_label': 'DATE'}
{'patient_id': 9, 'sentence': 'No se identifica infiltración tumoral.', 'NE

# 🚫 Carga del modelo de detección de negación e incertidumbre

Se carga el modelo `JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES`, diseñado para detectar expresiones de **negación** y **incertidumbre diagnóstica** en textos clínicos.

Este modelo se utiliza mediante el pipeline de `transformers` con estrategia de agregación `"simple"`, que agrupa subpalabras en entidades completas.

El pipeline resultante, `neg_pipe`, permitirá analizar cada oración clínica para determinar si una entidad está expresada de forma afirmativa, negada o con incertidumbre.


In [108]:
# 0. Cargar modelo de negación / incertidumbre

neg_model_id = "JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES"

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
neg_pipe = pipeline(
    "ner",
    model     = AutoModelForTokenClassification.from_pretrained(neg_model_id),
    tokenizer = AutoTokenizer.from_pretrained(neg_model_id),
    aggregation_strategy="simple"
)


Device set to use cuda:0


# 🗂️ Mapeo manual de etiquetas del modelo de negación

El modelo de negación/incertidumbre devuelve etiquetas genéricas (`LABEL_0` a `LABEL_8`) que no tienen una interpretación explícita. Por ello, se realiza un **mapeo manual** (`label2nombre`) para asignar a cada etiqueta un nombre representativo y funcional.

Además, se definen dos conjuntos clave:

- `NEGADAS`: agrupa las etiquetas asociadas a expresiones de **negación** y a entidades negadas.
- `INCIERTAS`: agrupa las etiquetas que indican **incertidumbre diagnóstica** o ambigüedad en el texto.

Este mapeo es esencial para clasificar correctamente el estado final de cada entidad extraída (Afirmativa, Negada o Sospechosa) en los pasos posteriores.


In [120]:
# 1. Mapeo manual y conjuntos para decidir el Estado

label2nombre = {
    "LABEL_0": "CUE_NEG",
    "LABEL_1": "B-NEG_ENT",
    "LABEL_2": "B-UNC_CUE",
    "LABEL_3": "B-UNC_ENT",
    "LABEL_4": "LINK_NEG",
    "LABEL_5": "I-NEG_ENT",
    "LABEL_6": "I-UNC_CUE",
    "LABEL_7": "I-UNC_ENT",
    "LABEL_8": "O"
}

NEGADAS   = {"LABEL_0", "LABEL_1", "LABEL_4", "LABEL_5"}
INCIERTAS = {"LABEL_2", "LABEL_3", "LABEL_6", "LABEL_7"}


# ✴️ Heurística para determinar el Estado de la entidad

Se define la función `estado_entidad_heur` que clasifica cada entidad extraída como **Afirmativa**, **Negada** o **Sospechosa**, utilizando los resultados del modelo de negación.

Durante el desarrollo se identificaron casos donde una misma oración contenía múltiples entidades con distintos estados (por ejemplo, una afirmada y otra negada). Para mitigar estos errores, se implementó una **heurística basada en puntuación** (`PUNTOS`), que delimita los **ámbitos** de las expresiones de negación o incertidumbre.

### Lógica de la heurística:

1. **Cue dentro de la entidad:**  
   Si un cue de negación (`CUE_NEG`) o incertidumbre (`UNC_CUE`) está **dentro del span de la entidad**, se clasifica directamente como `Negada` o `Sospechosa`.

2. **Cue antes de la entidad y mismo ámbito:**  
   Si el cue aparece antes de la entidad pero **antes del siguiente signo de puntuación**, se asume que afecta a la entidad:
   - `Negada` si el cue es de negación.
   - `Sospechosa` si es de incertidumbre.

3. **Por defecto:**  
   Si no se encuentra ningún cue relevante, se clasifica como `Afirmativa`.

Esta función es clave para **asociar correctamente el contexto lingüístico** a cada entidad identificada en las historias clínicas.


In [136]:
import re, string

PUNTOS    = {",", ";", ":", "."}
NEG_CUES  = {"LABEL_0"}              # CUE_NEG
UNC_CUES  = {"LABEL_2", "LABEL_6"}   # cues de incertidumbre

def estado_entidad_heur(ent, tokens_neg, sentence):
    e_start, e_end = ent["start"], ent["end"]

    # 0) ¿Hay un cue DENTRO de la entidad?
    for tok in tokens_neg:
        lab = tok.get("entity_group") or tok["entity"]
        if lab in NEG_CUES and tok["start"] >= e_start and tok["end"] <= e_end:
            return "Negada"
        if lab in UNC_CUES and tok["start"] >= e_start and tok["end"] <= e_end:
            return "Sospechosa"

    # helper: próxima puntuación tras un índice
    def next_punct(pos):
        for idx in range(pos, len(sentence)):
            if sentence[idx] in PUNTOS:
                return idx
        return len(sentence)

    # 1) CUE_NEG antes de la entidad y mismo ámbito
    for tok in tokens_neg:
        lab = tok.get("entity_group") or tok["entity"]
        if lab in NEG_CUES:
            cue_end   = tok["end"]
            scope_end = next_punct(cue_end)
            if e_start >= cue_end and e_end <= scope_end:
                return "Negada"

    # 2) CUE_UNC antes de la entidad y mismo ámbito
    for tok in tokens_neg:
        lab = tok.get("entity_group") or tok["entity"]
        if lab in UNC_CUES:
            cue_end   = tok["end"]
            scope_end = next_punct(cue_end)
            if e_start >= cue_end and e_end <= scope_end:
                return "Sospechosa"

    # 3) Por defecto
    return "Afirmativa"


# 🧠 Integración de entidades con clasificación de Estado

Se recorre cada oración de cada paciente y se integran ambos modelos (NER e identificación de negación/incertidumbre) para construir la tabla final estructurada.

### Proceso:

1. **Extracción de entidades clínicas:**  
   Se aplica el modelo NER a la oración para identificar entidades como biomarcadores, diagnósticos, tratamientos, etc.

2. **Detección de tokens de negación e incertidumbre:**  
   Se aplica el segundo modelo para obtener los cues lingüísticos que indican **negación** o **incertidumbre** (como “no”, “ausencia”, “probable”, “sugestivo”, etc.).

3. **Clasificación contextual de cada entidad:**  
   Para **cada entidad encontrada**, se llama a la función `estado_entidad_heur`, que evalúa su contexto y determina si está:
   - `Afirmativa`
   - `Negada`
   - `Sospechosa`

4. **Construcción de la base estructurada:**  
   Por cada entidad extraída se guarda una fila con:
   - `patient_id`: Identificador del archivo procesado.
   - `sentence`: La oración completa donde se encontró la entidad.
   - `NER`: El texto de la entidad.
   - `NER_label`: La categoría semántica asignada por el modelo NER.
   - `Estado`: Resultado de la clasificación contextual (negada, sospechosa, afirmativa).

Este paso permite obtener una base final lista para exportar y analizar en formatos estructurados como CSV o DataFrame de Pandas.


In [137]:
filas_final = []

for patient_id, oraciones in historias.items():
    for sentence in oraciones:

        # 1. entidades clínicas con spans
        ents = ner_pipe(sentence)
        if not ents:
            continue

        # 2. tokens del modelo de negación (también traen spans)
        tok_neg = neg_pipe(sentence)

        # 3. clasifica CADA entidad por separado
        for ent in ents:
            filas_final.append({
                "patient_id": patient_id,
                "sentence":   sentence,
                "NER":        ent["word"],
                "NER_label":  ent.get("entity_group") or ent.get("entity"),
                "Estado":     estado_entidad_heur(ent, tok_neg, sentence)
            })


In [174]:
for fila in filas_final:
    if fila['patient_id'] == 4:
        print(fila)

{'patient_id': 4, 'sentence': 'Carcinoma ductal infiltrante de mama derecha G2/3, pT2 pN1a cM0.', 'NER': 'Carcinoma ductal infiltrante de mama derecha', 'NER_label': 'CANCER_CONCEPT', 'Estado': 'Afirmativa'}
{'patient_id': 4, 'sentence': 'Carcinoma ductal infiltrante de mama derecha G2/3, pT2 pN1a cM0.', 'NER': 'G2/3', 'NER_label': 'STAGE', 'Estado': 'Afirmativa'}
{'patient_id': 4, 'sentence': 'Carcinoma ductal infiltrante de mama derecha G2/3, pT2 pN1a cM0.', 'NER': 'pT2 pN1a cM0', 'NER_label': 'TNM', 'Estado': 'Afirmativa'}
{'patient_id': 4, 'sentence': 'Carcinoma ductal infiltrante de mama derecha G2/3, pT2 pN1a cM0.', 'NER': 'Carcinoma ductal infiltrante de mama derecha', 'NER_label': 'CANCER_CONCEPT', 'Estado': 'Afirmativa'}
{'patient_id': 4, 'sentence': 'Carcinoma ductal infiltrante de mama derecha G2/3, pT2 pN1a cM0.', 'NER': 'G2/3', 'NER_label': 'STAGE', 'Estado': 'Afirmativa'}
{'patient_id': 4, 'sentence': 'Carcinoma ductal infiltrante de mama derecha G2/3, pT2 pN1a cM0.', 'NE

In [141]:
import pandas as pd
df = pd.DataFrame(filas_final)
ruta_csv = "/content/drive/MyDrive/Analitica en Salud/hist_clinicas_ner_neg.csv"
df.to_csv(ruta_csv, index=False, encoding="utf-8")
print(f"✅ CSV final con Estado por entidad guardado en: {ruta_csv}")

✅ CSV final con Estado por entidad guardado en: /content/drive/MyDrive/Analitica en Salud/hist_clinicas_ner_neg.csv
